In [30]:
import numpy as np 
import math
import seaborn as sns
import tensorflow as tf
from numpy.ma.core import ceil
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import animation, colors
from sklearn import preprocessing as sk_preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import confusion_matrix
import cv2

from keras.layers import Dropout
from keras.layers.convolutional import Convolution2D
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras import backend as K


In [ ]:
df = pd.read_csv('/Kidney_data.csv')
df.describe()

,id,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo
count,400.000000,391.000000,388.000000,353.000000,354.000000,351.000000,356.000000,381.000000,383.000000,313.000000,312.000000,348.000000
mean,199.500000,51.483376,76.469072,1.017408,1.016949,0.450142,148.036517,57.425722,3.072454,137.528754,4.627244,12.526437
std,115.614301,17.169714,13.683637,0.005717,1.352679,1.099191,79.281714,50.503006,5.741126,10.408752,3.193904,2.912587
min,0.000000,2.000000,50.000000,1.005000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000
25%,99.750000,42.000000,70.000000,1.010000,0.000000,0.000000,99.000000,27.000000,0.900000,135.000000,3.800000,10.300000
50%,199.500000,55.000000,80.000000,1.020000,0.000000,0.000000,121.000000,42.000000,1.300000,138.000000,4.400000,12.650000
75%,299.250000,64.500000,80.000000,1.020000,2.000000,0.000000,163.000000,66.000000,2.800000,142.000000,4.900000,15.000000
max,399.000000,90.000000,180.000000,1.025000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000


In [ ]:
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [ ]:
df.info() #(400, 26) and display null values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [ ]:
df.isnull()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
396,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
397,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
398,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.isnull().sum() #no of null

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.to_csv('/content/Kidney_data_One.csv', index=False) #save new data into kidney_data 

# Model One: CNN on Non-NULL Dataset


In [ ]:
df = pd.read_csv('/content/Kidney_data_One.csv')

In [ ]:
# Read training and test data files
train = pd.read_csv("/content/Kidney_data_One.csv").values
test  = pd.read_csv("//content/Kidney_data_One.csv").values # dimensions are (158, 26)

In [ ]:

print(train.shape)
print(test.shape)


(158, 26)
(158, 26)


In [ ]:

# Load train and test data
train = np.random.rand(158, 26) # Example train data
test = np.random.rand(158, 26) # Example test data

# Pad the array with zeros to make it divisible by 784
train_padded = np.pad(train, ((0,0), (0, 784 - 26)), 'constant', constant_values=0)
test_padded = np.pad(test, ((0,0), (0, 784 - 26)), 'constant', constant_values=0)

# Reshape and normalize train data
trainX = train_padded[:, :784].reshape(train.shape[0], 784,1).astype('float32')
X_train = trainX / 255.0
y_train = train[:, 0]

# Reshape and normalize test data
testX = test_padded[:, :784].reshape(test.shape[0], 784,1).astype('float32')
X_test = testX / 255.0
y_test = test[:, 0]

print ('X_train')
print ('y_train')


X_train
y_train


In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

y_train = y_train.astype('int')
y_test = y_test.astype('int')

y_train = lb.fit_transform(y_train)
y_test = lb.fit_transform(y_test)


In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(784,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
5/5 [==============================] - 1s 36ms/step - loss: 0.6642 - accuracy: 1.0000
Epoch 2/10
5/5 [==============================] - 0s 34ms/step - loss: 0.3995 - accuracy: 1.0000
Epoch 3/10
5/5 [==============================] - 0s 39ms/step - loss: 0.0854 - accuracy: 1.0000
Epoch 4/10
5/5 [==============================] - 0s 32ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 5/10
5/5 [==============================] - 0s 38ms/step - loss: 6.7290e-04 - accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 0s 33ms/step - loss: 1.1749e-04 - accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 0s 32ms/step - loss: 3.5927e-05 - accuracy: 1.0000
Epoch 8/10
5/5 [==============================] - 0s 33ms/step - loss: 1.6549e-05 - accuracy: 1.0000
Epoch 9/10
5/5 [==============================] - 0s 31ms/step - loss: 1.0065e-05 - accuracy: 1.0000
Epoch 10/10
5/5 [==============================] - 0s 34ms/step - loss: 7.3638e-06 - accuracy: 1.0000


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 782, 64)           256       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 391, 64)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 25024)             0         
                                                                 
 dense (Dense)               (None, 2)                 50050     
                                                                 
Total params: 50,306
Trainable params: 50,306
Non-trainable params: 0
_________________________________________________________________


A Conv1D layer with 64 filters and a kernel size of 3, which outputs a tensor of shape (None, 782, 64). The number of trainable parameters in this layer is 256, which corresponds to the number of filter weights plus bias terms.

A MaxPooling1D layer that performs max pooling with a pool size of 2 and a stride of 2, which outputs a tensor of shape (None, 391, 64). This layer has no trainable parameters.

A Flatten layer that flattens the output tensor of the previous layer into a one-dimensional tensor of shape (None, 25024). This layer has no trainable parameters.

A Dense layer with 2 output units, which outputs a tensor of shape (None, 2). The number of trainable parameters in this layer is 50,050, which corresponds to the weights and bias terms of the dense layer.

In [ ]:
y_pred = model.predict(X_test).argmax(axis=1)

accuracy = accuracy_score(y_test, y_pred)
print('Test accuracy:', accuracy)

5/5 [==============================] - 0s 11ms/step
Test accuracy: 1.0


# Model Two: CNN on Null values Dataset Using Bayes (GaussianNB)

In [52]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
df = pd.read_csv('/Kidney_data.csv')

In [53]:
# remove this
d= df['classification'].replace({'ckd': 0, 'notckd': 1}) 
# Yet, there is one data point that is misclassified and that is why the confusion matrix returns as a 3 dimensional matrix instead of two 

In [54]:
#Transform the non numeric data in the columns
for column in df.columns:
  if df[column].dtype==np.number:
    continue
  else:
    df[column]=LabelEncoder().fit_transform(df[column])

<ipython-input-54-918540d8aceb>:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype==np.number:
<ipython-input-54-918540d8aceb>:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype==np.number:
<ipython-input-54-918540d8aceb>:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype==np.number:
<ipython-input-54-918540d8aceb>:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype==np.number:
<ipython-input-54-918540d8aceb>:3: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. Th

In [55]:
gnb = GaussianNB()

In [56]:
for col in df.columns:
    if df[col].isna().sum() > 0:
        mask = df[col].isna()
        train_data = df.loc[~mask].dropna(axis=1)
        test_data = df.loc[mask, train_data.columns]
        train_target = train_data[col]
        mean = train_target.mean()
        std = train_target.std()
        train_target_cat = pd.cut(train_target, bins=[-np.inf, mean-std, mean+std, np.inf], labels=[0,1,2])
        
        train_data = train_data.drop(columns=[col])
        test_data = test_data.drop(columns=[col])
        imputed_values = gnb.fit(train_data, train_target_cat).predict(test_data)
        df.loc[mask, col] = imputed_values
df.isnull().sum()

id                0
age               0
bp                0
sg                0
al                0
su                0
rbc               0
pc                0
pcc               0
ba                0
bgr               0
bu                0
sc                0
sod               0
pot               0
hemo              0
pcv               0
wc                0
rc                0
htn               0
dm                0
cad               0
appet             0
pe                0
ane               0
classification    0
dtype: int64

In [57]:
#remove this
df['classification'].head(400)

0      0
1      0
2      0
3      0
4      0
      ..
395    2
396    2
397    2
398    2
399    2
Name: classification, Length: 400, dtype: int64

In [58]:
# Split the data into training and testing sets
A = df.drop(columns=['classification'], axis=1).values
B = df['classification']


A = np.reshape(A, (A.shape[0], 25, 1))

A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3)

#remove it 
print(B.dtype)


int64


In [59]:
#remove it 
print(A_train)

[[[326.]
  [ 47.]
  [ 60.]
  ...
  [  0.]
  [  0.]
  [  0.]]

 [[129.]
  [ 75.]
  [ 70.]
  ...
  [  1.]
  [  1.]
  [  0.]]

 [[353.]
  [ 39.]
  [ 60.]
  ...
  [  0.]
  [  0.]
  [  0.]]

 ...

 [[184.]
  [ 54.]
  [ 60.]
  ...
  [  1.]
  [  1.]
  [  0.]]

 [[364.]
  [ 73.]
  [ 80.]
  ...
  [  0.]
  [  0.]
  [  0.]]

 [[378.]
  [ 71.]
  [ 60.]
  ...
  [  0.]
  [  0.]
  [  0.]]]


In [60]:
#Build and Define the CNN model
model1 = Sequential()
model1.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(25, 1)))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Flatten())
model1.add(Dense(3, activation='softmax'))
model1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
# Train the model
model1.fit(A_train, B_train, epochs=30, batch_size=16, validation_data=(A_test, B_test))

Epoch 1/30
18/18 [==============================] - 0s 5ms/step - loss: 0.0195 - accuracy: 0.9929 - val_loss: 0.1724 - val_accuracy: 0.9750
Epoch 2/30
18/18 [==============================] - 0s 3ms/step - loss: 0.0258 - accuracy: 0.9929 - val_loss: 0.1954 - val_accuracy: 0.9667
Epoch 3/30
18/18 [==============================] - 0s 3ms/step - loss: 0.0200 - accuracy: 0.9964 - val_loss: 0.1989 - val_accuracy: 0.9667
Epoch 4/30
18/18 [==============================] - 0s 3ms/step - loss: 0.0250 - accuracy: 0.9929 - val_loss: 0.1853 - val_accuracy: 0.9667
Epoch 5/30
18/18 [==============================] - 0s 4ms/step - loss: 0.0152 - accuracy: 0.9964 - val_loss: 0.1860 - val_accuracy: 0.9667
Epoch 6/30
18/18 [==============================] - 0s 3ms/step - loss: 0.0162 - accuracy: 0.9964 - val_loss: 0.1816 - val_accuracy: 0.9667
Epoch 7/30
18/18 [==============================] - 0s 5ms/step - loss: 0.0128 - accuracy: 1.0000 - val_loss: 0.2010 - val_accuracy: 0.9667
Epoch 8/30
18/18 [==

In [64]:
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Predict classes for test set
y_pred = model1.predict(A_test)

y_pred_classes = np.argmax(y_pred, axis=1)

# Compute accuracy
accuracy = accuracy_score(B_test, y_pred_classes)
print("Accuracy:", accuracy)

# Compute confusion matrix
conf_mat = confusion_matrix(B_test, y_pred_classes)
print("Confusion matrix:\n", conf_mat)


4/4 [==============================] - 0s 2ms/step
Accuracy: 0.975
Confusion matrix:
 [[78  0  2]
 [ 1  0  0]
 [ 0  0 39]]


 # Model Three: CNN on Null values Dataset Using K-nearest

In [2]:
from sklearn.impute import KNNImputer
import pandas as pd

df = pd.read_csv('/Kidney_data.csv')

In [3]:
df.isna().sum()

id                  0
age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [4]:
df.columns

Index(['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [5]:
cat_variables = df[ ['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane']]

#cat_variables = df["age", "bp","sg","al","su", "rbc", "pc", "pcc", "ba","bgr","bu","sc","sod","pot","hemo","pcv","wc","rc" "htn", "dm", "cad", "appet", "pe", "ane"]
cat_dummies = pd.get_dummies(cat_variables, drop_first=True)
cat_dummies.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,htn_yes,dm_\tyes,dm_ yes,dm_no,dm_yes,cad_no,cad_yes,appet_poor,pe_yes,ane_yes
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,...,1,0,0,0,1,1,0,0,0,0
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,...,0,0,0,1,0,1,0,0,0,0
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,...,0,0,0,0,1,1,0,1,0,1
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,1,0,0,1,0,1,0,1,1,1
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,...,0,0,0,1,0,1,0,0,0,0


In [31]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# Select only numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Scale the numeric columns
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [ ]:
"""
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

#This code will first select only the numeric columns using the select_dtypes method and the np.number parameter. 
#It will then apply the fit_transform method to only the numeric columns.

"""


In [ ]:
"""
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
df.head()
"""


'\nfrom sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\ndf = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)\ndf.head()\n'

In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np

# Select only numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Impute missing values using KNNImputer
imputer = KNNImputer(n_neighbors=5)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Scale the numeric columns
scaler = MinMaxScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [33]:
df.isna().any()

id                False
age               False
bp                False
sg                False
al                False
su                False
rbc                True
pc                 True
pcc                True
ba                 True
bgr               False
bu                False
sc                False
sod               False
pot               False
hemo              False
pcv                True
wc                 True
rc                 True
htn                True
dm                 True
cad                True
appet              True
pe                 True
ane                True
classification    False
dtype: bool

In [34]:
df.isna().sum()


id                  0
age                 0
bp                  0
sg                  0
al                  0
su                  0
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                 0
bu                  0
sc                  0
sod                 0
pot                 0
hemo                0
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [35]:
#remove this
df['classification'].head(400)

0         ckd
1         ckd
2         ckd
3         ckd
4         ckd
        ...  
395    notckd
396    notckd
397    notckd
398    notckd
399    notckd
Name: classification, Length: 400, dtype: object

In [11]:
# Split the data into training and testing sets
A = df.drop(columns=['classification'], axis=1).values
B = df['classification']


A = np.reshape(A, (A.shape[0], 25, 1))

A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3)

#remove it 
print(B.dtype)


object


In [36]:
#remove it 
print(A_train)

[[0.3333333333333333 0.7727272727272728 0.38461538461538464 ... 'good'
  'no' 'no']
 [0.794486215538847 0.6363636363636364 0.1538461538461539 ... 'good' 'no'
  'no']
 [0.24561403508771928 0.5454545454545455 0.6923076923076925 ... 'poor'
  'no' 'yes']
 ...
 [0.42105263157894735 0.7159090909090909 0.1538461538461539 ... 'good'
  'no' 'no']
 [0.6967418546365914 0.5227272727272727 0.07692307692307693 ... 'good'
  'no' 'no']
 [0.8596491228070176 0.39772727272727276 0.07692307692307693 ... 'good'
  'no' 'no']]


In [45]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Load dataset
df = pd.read_csv('/Kidney_data.csv')



# Replace missing values with NaN
df.replace('?', np.nan, inplace=True)
df = pd.get_dummies(df)

# Split the dataset into features and labels
T = df.iloc[:, :-1].values
U = df.iloc[:, -1].values



# Split the dataset into training and testing sets
T_train, T_test, U_train, U_test = train_test_split(T, U, test_size=0.3, random_state=42)

# Handle missing values using KNN imputation
imputer = KNNImputer(n_neighbors=5)
T_train = imputer.fit_transform(T_train)
T_test = imputer.transform(T_test)

# Scale the features
scaler = StandardScaler()
T_train = scaler.fit_transform(T_train)
T_test = scaler.transform(T_test)

# Reshape the data to match the input shape of the CNN
T_train = T_train.reshape(T_train.shape[0], T_train.shape[1], 1)
T_test = T_test.reshape(T_test.shape[0], T_test.shape[1], 1)

# Convert labels to categorical format using one-hot encoding
num_classes = 2
U_train_categorical = tf.keras.utils.to_categorical(U_train, num_classes)
U_test_categorical = tf.keras.utils.to_categorical(U_test, num_classes)


# Define the CNN model
model3 = Sequential()
model3.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(Flatten())
model3.add(Dense(16, activation='relu'))
model3.add(Dense(2, activation='softmax'))

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [46]:
# Train the model
model3.fit(T_train, U_train_categorical, epochs=10, batch_size=32, validation_data=(T_test, U_test_categorical))

Epoch 1/10
9/9 [==============================] - 1s 63ms/step - loss: 0.5901 - accuracy: 0.6286 - val_loss: 0.4634 - val_accuracy: 0.7000
Epoch 2/10
9/9 [==============================] - 0s 24ms/step - loss: 0.3566 - accuracy: 0.8643 - val_loss: 0.2353 - val_accuracy: 0.9667
Epoch 3/10
9/9 [==============================] - 0s 32ms/step - loss: 0.1721 - accuracy: 0.9607 - val_loss: 0.1496 - val_accuracy: 0.9667
Epoch 4/10
9/9 [==============================] - 0s 16ms/step - loss: 0.1041 - accuracy: 0.9607 - val_loss: 0.1106 - val_accuracy: 0.9667
Epoch 5/10
9/9 [==============================] - 0s 14ms/step - loss: 0.0842 - accuracy: 0.9679 - val_loss: 0.1160 - val_accuracy: 0.9750
Epoch 6/10
9/9 [==============================] - 0s 20ms/step - loss: 0.0624 - accuracy: 0.9750 - val_loss: 0.0950 - val_accuracy: 0.9750
Epoch 7/10
9/9 [==============================] - 0s 16ms/step - loss: 0.0438 - accuracy: 0.9857 - val_loss: 0.0903 - val_accuracy: 0.9750
Epoch 8/10
9/9 [===========

In [51]:
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# Predict classes for test set
U_pred = model3.predict(T_test)

U_pred_classes = np.argmax(U_pred, axis=1)

# Compute accuracy
accuracy = accuracy_score(U_test, U_pred_classes)
print("Accuracy:", accuracy)

# Compute confusion matrix
conf_mat = confusion_matrix(U_test, U_pred_classes)
print("Confusion matrix:\n", conf_mat)

4/4 [==============================] - 0s 5ms/step
Accuracy: 0.9833333333333333
Confusion matrix:
 [[74  2]
 [ 0 44]]


For the CNN model with Bayesian regularization, the confusion matrix shows that:

Class 0 was correctly classified 78 times (TP), incorrectly classified as class 2 twice (FP), and never classified as class 1 (TN = 0, FN = 0).
Class 1 was never correctly classified (TP = 0), incorrectly classified as class 0 once (FP), and never classified as class 2 (TN = 39, FN = 0).
Class 2 was correctly classified 39 times (TP), never classified as class 0 (TN = 0, FN = 0), and never classified as class 1 (TN = 0, FN = 0).
Overall, the model achieved an accuracy of 0.975, which means that it correctly classified 97.5% of the samples.

For the k-NN model, the confusion matrix shows that:

Class 0 was correctly classified 74 times (TP) and incorrectly classified as class 1 twice (FP).
Class 1 was correctly classified 44 times (TP) and never classified as class 0 (TN = 0, FN = 0).
Overall, the model achieved an accuracy of 0.983, which means that it correctly classified 98.3% of the samples.